In [2]:
import pandas as pd
import numpy as np
import json
import joblib
from notebooks.utils.load_ckpt import define_model

pd.set_option('max_colwidth',1000)

import os
os.getcwd()
os.chdir('/home/hyeryung_son/mucoco/')

def merge_lists(lists):
    a = []
    for _list in lists:
        a.append(_list)
        

In [3]:
# load model, tokenizer
model, tokenizer = define_model('models/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/checkpoint_best/pytorch_model.bin')

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2-large and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
S

DEVICE:  cuda


In [4]:
dsource = "gpt2"

# load test set files
input_id_file = joblib.load(f'outputs/toxicity/save-init-gen-all-uniform/testset_FINAL_{dsource}_input_ids.pkl')
gt_file = joblib.load(f'outputs/toxicity/save-init-gen-all-uniform/testset_FINAL_{dsource}_labels.pkl')

In [5]:
# fprefix = f"outputs/toxicity/locate-edit-{dsource}/outputs_epsilon-5"
fprefix = f"/home/hyeryung_son/mucoco/outputs/toxicity/locate-edit-{dsource}-loc-6toks--1steps-project-1steps/outputs_epsilon-5"

# load intermediate predictions file
pred_file = pd.read_json(f'{fprefix}.txt.intermediate', lines=True)
pred_file["indices"] = pred_file["step_0_indices"]
pred_file['indices'] = pred_file['indices'].apply(lambda x: set(x[0]))

In [34]:
print("check if prediction file length equal to test file length")
assert len(pred_file) == len(input_id_file)
assert len(pred_file) == len(gt_file)

check if prediction file length equal to test file length


In [6]:
pred_file['input_ids'] = input_id_file

pred_file['gt_gt0'] = [set(np.where(np.array(x) > 0)[0]) for x in gt_file]
pred_file['gt_eq1'] = [set(np.where(np.array(x) == 1.0)[0]) for x in gt_file]


In [7]:
## EDA locate length => what's average # of located tokens
pred_file['len_gt_gt0'] = pred_file['gt_gt0'].apply(len)
pred_file['len_gt_eq1'] = pred_file['gt_eq1'].apply(len)

print(pred_file['len_gt_gt0'].describe())

count    115.000000
mean       5.286957
std        4.279214
min        0.000000
25%        2.000000
50%        5.000000
75%        7.500000
max       24.000000
Name: len_gt_gt0, dtype: float64


In [37]:
pred_file["indices"].apply(len).describe()

count    115.000000
mean       5.104348
std        2.552515
min        0.000000
25%        4.000000
50%        6.000000
75%        6.000000
max       18.000000
Name: indices, dtype: float64

In [38]:
# fprefix = f"outputs/toxicity/locate-edit-{dsource}/outputs_epsilon-5"
fprefix = f"/home/hyeryung_son/mucoco/outputs/toxicity/locate-edit-{dsource}/old_locate_function/outputs_epsilon-5"

# load intermediate predictions file
pred_file2 = pd.read_json(f'{fprefix}.txt.intermediate', lines=True)
# pred_file2["indices"] = pred_file2["step_0_indices"]
pred_file2['indices'] = pred_file2['indices'].apply(lambda x: set(x[0]))
pred_file2["indices"].apply(len).describe()

count    115.000000
mean       4.730435
std        1.237724
min        0.000000
25%        4.000000
50%        5.000000
75%        6.000000
max        6.000000
Name: indices, dtype: float64

In [151]:
## 눈으로 확인했을 때는, 뭐가 더 낫다 라고 하기가 어려웠다. 다만, gt_gt0 라벨의 경우 a, the 같은 관사가 포함되어 있는데
## 이것들은 굳이 필요가 없을것 같기도 하다.

pred_file['gt_words_gt0'] = pred_file[['input_ids', 'gt_gt0']].apply(lambda x: tokenizer.decode(np.array(x[0])[sorted(list(x[1]))]), axis=1)
pred_file['gt_words_eq1'] = pred_file[['input_ids', 'gt_eq1']].apply(lambda x: tokenizer.decode(np.array(x[0])[sorted(list(x[1]))]), axis=1)

In [152]:
## precision, recall
suffix = 'gt0'
pred_indices = pred_file['indices'].tolist()
gt_indices = pred_file[f"gt_{suffix}"].tolist()

precision_list = []
recall_list = []
for i in range(len(pred_indices)):
    intersection = pred_indices[i].intersection(gt_indices[i])
    if len(gt_indices[i]) == 0:
        recall = np.nan
    else:
        recall = len(intersection) / len(gt_indices[i])
    if len(pred_indices[i]) == 0:
        precision = np.nan
    else:
        precision = len(intersection) / len(pred_indices[i])
        
    precision_list.append(precision)
    recall_list.append(recall)

pred_file[f'precision_{suffix}'] = precision_list
pred_file[f'recall_{suffix}'] = recall_list

In [153]:
suffix = 'eq1'
pred_indices = pred_file['indices'].tolist()
gt_indices = pred_file[f"gt_{suffix}"].tolist()

precision_list = []
recall_list = []
for i in range(len(pred_indices)):
    intersection = pred_indices[i].intersection(gt_indices[i])
    if len(gt_indices[i]) == 0:
        recall = np.nan
    else:
        recall = len(intersection) / len(gt_indices[i])
    if len(pred_indices[i]) == 0:
        precision = np.nan
    else:
        precision = len(intersection) / len(pred_indices[i])
        
    precision_list.append(precision)
    recall_list.append(recall)

pred_file[f'precision_{suffix}'] = precision_list
pred_file[f'recall_{suffix}'] = recall_list

In [154]:
suffix = 'gt0'
print(f"precision_{suffix} : {pred_file[f'precision_{suffix}'].mean():.4f}")
print(f"recall_{suffix} : {pred_file[f'recall_{suffix}'].mean():.4f}")

precision_gt0 : 0.3910
recall_gt0 : 0.6004


In [155]:
suffix = 'eq1'
print(f"precision_{suffix} : {pred_file[f'precision_{suffix}'].mean():.4f}")
print(f"recall_{suffix} : {pred_file[f'recall_{suffix}'].mean():.4f}")

precision_eq1 : 0.2652
recall_eq1 : 0.6990


In [156]:
fprefix

'/home/hyeryung_son/mucoco/outputs/toxicity/locate-edit-jigsaw-loc-6toks--1steps-project-1steps/outputs_epsilon-5'

In [157]:
with open(f'{fprefix}.eval', 'w') as f:
    f.writelines("====================================\n")
    suffix = 'gt0'
    f.writelines(f"precision_{suffix} : {pred_file[f'precision_{suffix}'].mean():.4f}\n")
    f.writelines(f"recall_{suffix} : {pred_file[f'recall_{suffix}'].mean():.4f}\n")
    f.writelines("====================================\n")
    suffix = 'eq1'
    f.writelines(f"precision_{suffix} : {pred_file[f'precision_{suffix}'].mean():.4f}\n")
    f.writelines(f"recall_{suffix} : {pred_file[f'recall_{suffix}'].mean():.4f}\n")
    f.writelines("====================================\n")